In [376]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


Trend line of what yLUNA is being used for?  PRISM Farm, yLUNA Staking, LPing, or Nothing.

In [377]:
prism_addr = 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw'
yluna_addr = 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'
pluna_addr = 'terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2'
pLuna_PRISM_Pair = 'terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs'
yLuna_PRISM_Pair = 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'

In [378]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [379]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [380]:
class GenericDataProvider():
    def __init__(self, claim, get_url, config , path_to_data='../data'):
        self.config = config
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
    
    def load(self):
        self.data = {}
        for name, c in self.config.items():
            current_data = {}
            df = self.claim(c['hash'])
            df.columns = [c.lower() for c in df.columns]
            current_data['from_hash'] = df
            try:
                df = pd.read_csv(f'{self.path_to_data}/{c["file_name"]}', index_col=0)
                df.columns = [c.lower() for c in df.columns]
                current_data['from_file'] = df
            except Exception as e:
                print(e)
            self.data[name] = current_data
        
    def standard_parse(self, df):
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        df['hr'] = df.block_timestamp.str[:-5] + '00:00.000'
        df['day'] = df.block_timestamp.str[:-9]
        return df
        

In [381]:
 config = {
     'transfers_prism': {'hash':'9651b671-1f85-4133-8384-20042dc1adb1',
                  'url':'',
                  'file_name':'transfers.csv'},
     'transfers_xprism' : {
                  'hash':'483df27b-4c69-47ec-8797-332e6c75ae86',
                  'url':'',
                  'file_name':'transfers_xprism.csv'},
     'boost_activation': {'hash':'ccbaf92f-6aaa-4460-a65f-83814431fd17',
                  'url':'',
                  'file_name':'boost_activation.csv'},
     'prism_forge': {
                  'hash':'2a364c52-9c37-4540-81c7-e5b890f0ef6b',
                  'url':'',
                  'file_name':'prism_forge.csv'},
     'amps_bound_unbound':{
                  'hash':'160d6d2c-487b-4173-af2e-ff51f9b64c0f',
                  'url':'',
                  'file_name':'amps_bound_unbound.csv'},
     'prism_stake':{
                  'hash':'7403d8c8-1687-44d3-9090-d72eec1407bc',
                  'url':'',
                  'file_name':'prism_stake.csv'}
}

In [382]:
swaps_config = {
     'swaps':{
                  'hash':'1bfd8019-89a1-470d-8868-60d71e57d1d0',
                  'url':'',
                  'file_name':'swaps.csv'},
     'router':{
                  'hash':'69b149fb-81ba-4860-aac1-b17f0d6d7688',
                  'url':'',
                  'file_name':'router.csv'}
}

In [383]:
class SwapsDataProvider(GenericDataProvider):
    def load(self):
        super().load()
        self.swaps_df = self.data['swaps']['from_hash']
        self.swaps_df_from_csv = self.data['swaps']['from_file']
        self.swaps_df = self.swaps_df.append(self.swaps_df_from_csv).drop_duplicates(ignore_index=True)
        self.swaps_df = self.standard_parse(self.swaps_df)
        #
        self.router_df = self.data['router']['from_hash']
        self.router_df_from_csv = self.data['router']['from_file']
        self.router_df = self.swaps_df.append(self.router_df_from_csv).drop_duplicates(ignore_index=True)
        self.router_df = self.standard_parse(self.router_df)
            
    def write_to_csv(self):
        self.swaps_df.to_csv(f'{self.path_to_data}/swaps.csv')
        self.router_df.to_csv(f'{self.path_to_data}/router.csv')
        
    def parse_simple_swaps(self):
        self.swaps_df.columns = [c.lower() for c in self.swaps_df.columns]
        swaps_df = self.swaps_df[self.swaps_df.ask_asset != '']
        swaps_df = swaps_df[swaps_df.sender.notna()]
        swaps_df_pol = swaps_df.rename(columns={'sender':'user','ask_asset':'asset_received','offer_asset':'asset_given'})
        swaps_df_pol = swaps_df_pol[['block_timestamp','tx_id','price','user','asset_received','return_amount','asset_given','offer_amount']]
        swaps_df_pol['operation'] = 'swap'
        self.swaps_df_pol = swaps_df_pol
        
    def parse_router(self):
        self.router_df.columns = [c.lower() for c in self.router_df.columns]
        router_df = self.router_df.rename(columns={'sender':'user'})
        router_df_1 = router_df[['block_timestamp','tx_id','0_ask_asset','0_offer_amount','0_offer_asset','0_price','0_return_amount','user']]
        router_df_1 = router_df_1.rename(columns={'0_ask_asset':'asset_received','0_offer_amount':'offer_amount',
                           '0_price':'price','0_return_amount':'return_amount','0_offer_asset':'asset_given'})
        router_df_2 = router_df[['block_timestamp','tx_id','1_ask_asset','1_offer_amount','1_offer_asset','1_price','1_return_amount','user']]
        router_df_2 = router_df_2.rename(columns={'1_ask_asset':'asset_received','1_offer_amount':'offer_amount',
                           '1_price':'price','1_return_amount':'return_amount','1_offer_asset':'asset_given'})
        router_df_pol = router_df_1.append(router_df_2)
        router_df_pol['operation'] = 'swap'
        self.router_df_pol = router_df_pol
    
    def parse(self):
        self.parse_simple_swaps()
        self.parse_router()
        self.swaps_df_all = self.router_df_pol.append(self.swaps_df_pol[self.router_df_pol.columns])
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.swaps_df_all['hr'] = self.swaps_df_all.block_timestamp.str[:-5] + '00:00.000'
        self.swaps_df_all['day'] = self.swaps_df_all.block_timestamp.str[:-9]
        #
        yluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='yLUNA')|(self.swaps_df_all.asset_received=='yLUNA')]
        yluna_swaps['amount_signed'] = yluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'yLUNA' else -row.return_amount,axis=1)
        yluna_swaps['type'] = 'swap'
        self.yluna_swaps = yluna_swaps
        #
        pluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='pLUNA')|(self.swaps_df_all.asset_received=='pLUNA')]
        pluna_swaps['amount_signed'] = pluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'pLUNA' else -row.return_amount,axis=1)
        pluna_swaps['type'] = 'swap'
        self.pluna_swaps = pluna_swaps
        #
        given = swaps_dp.swaps_df_all.copy()
        given['asset'] = given.asset_given
        given['amount_signed'] = -given.offer_amount

        received = swaps_dp.swaps_df_all.copy()
        received['asset'] = received.asset_received
        received['amount_signed'] = received.return_amount

        cols = ['block_timestamp', 'tx_id','user', 'hr',
               'day', 'asset', 'amount_signed']
        swaps_single = received[cols].append(given[cols])
        self.swaps_single = swaps_single[swaps_single.amount_signed.notna()]
        

In [384]:
class PrismBalanceDataProvider(GenericDataProvider):
    def load(self):
        super().load()
        self.transfers_df = self.data['transfers_prism']['from_hash']
        self.transfers_df_from_csv = self.data['transfers_prism']['from_file']
        self.transfers_df = self.transfers_df.append(self.transfers_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.transfers_xprism_df = self.data['transfers_xprism']['from_hash']
        self.transfers_xprism_df_from_csv = self.data['transfers_xprism']['from_file']
        self.transfers_xprism_df = self.transfers_df.append(self.transfers_xprism_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.boost_activation_df = self.data['boost_activation']['from_hash']
        self.boost_activation_df_from_csv = self.data['boost_activation']['from_file']
        self.boost_activation_df = self.boost_activation_df.append(self.boost_activation_df_from_csv).drop_duplicates(ignore_index=True)
        #self.boost_activation_df = self.standard_parse(self.boost_activation_df)
        #
        self.prism_forge_df = self.data['prism_forge']['from_hash']
        self.prism_forge_df_from_csv = self.data['prism_forge']['from_file']
        self.prism_forge_df = self.prism_forge_df.append(self.prism_forge_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.amps_bound_unbound_df = self.data['amps_bound_unbound']['from_hash']
        self.amps_bound_unbound_df_from_csv = self.data['amps_bound_unbound']['from_file']
        self.amps_bound_unbound_df = self.amps_bound_unbound_df.append(self.amps_bound_unbound_df_from_csv).drop_duplicates(ignore_index=True)
        #
        self.prism_stake_df = self.data['prism_stake']['from_hash']
        self.prism_stake_df_from_csv = self.data['prism_stake']['from_file']
        self.prism_stake_df = self.prism_stake_df.append(self.prism_stake_df_from_csv).drop_duplicates(ignore_index=True)
        
    def parse_forge_withdrawals(self):
        self.prism_forge_df = self.standard_parse(self.prism_forge_df)
        prism_forge_df = self.prism_forge_df
        prism_forge_df['amount'] = prism_forge_df.withdraw_amount/1000000
        prism_forge_df = prism_forge_df[['block_timestamp','tx_id','to_','amount','day']]
        prism_forge_df.columns = ['block_timestamp','tx_id','user','amount','day']
        prism_forge_df['amount_signed'] = prism_forge_df.amount
        self.prism_forge_df = prism_forge_df.drop_duplicates(ignore_index=True)
        
    def parse_transfers(self):
        self.transfers_df = self.standard_parse(self.transfers_df)
        self.transfers_df['amount'] = self.transfers_df.amount/1000000
        transfers_df = self.transfers_df
        send_tx = transfers_df[['block_timestamp','tx_id','from_','amount','day']]
        send_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        send_tx['amount_signed'] = -send_tx.amount
        receive_tx = transfers_df[['block_timestamp','tx_id','to_','amount','day']]
        receive_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        receive_tx['amount_signed'] = receive_tx.amount
        self.all_transfers = send_tx.append(receive_tx).drop_duplicates(ignore_index=True)
    
    def parse_transfers_xprism(self):
        self.transfers_xprism_df = self.standard_parse(self.transfers_xprism_df)
        self.transfers_xprism_df['amount'] = self.transfers_xprism_df.amount/1000000
        transfers_df = self.transfers_xprism_df
        send_tx = transfers_df[['block_timestamp','tx_id','from_','amount','day']]
        send_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        send_tx['amount_signed'] = -send_tx.amount
        receive_tx = transfers_df[['block_timestamp','tx_id','to_','amount','day']]
        receive_tx.columns = ['block_timestamp','tx_id','user','amount','day']
        receive_tx['amount_signed'] = receive_tx.amount
        self.all_transfers_xprism = send_tx.append(receive_tx).drop_duplicates(ignore_index=True)
        
    def parse_amps_bonding(self):
        self.amps_bound_unbound_df = self.standard_parse(self.amps_bound_unbound_df)
        amps_bound_unbound_df = self.amps_bound_unbound_df
        amps_bound_unbound_df['amount'] = amps_bound_unbound_df['amount']/1000000
        amps_bound_unbound_df['action'] = amps_bound_unbound_df.apply(lambda x: 'unbond' if x['0_action']=='transfer' else 'bond',axis=1)
        amps_bound_unbound_df['amount_signed'] = amps_bound_unbound_df.apply(lambda x: x.amount if x['action']=='unbond' else -x.amount,axis=1)
        amps_bound_unbound_df['user'] = amps_bound_unbound_df.apply(lambda x: x.to_ if x['action']=='unbond' else x.from_,axis=1)
        self.amps_bound_unbound_df = amps_bound_unbound_df[['block_timestamp', 'user', 'from_', 'hr',
               'msg_index', 'to_', 'tx_id', 'action', 'amount', 'day',
               'amount_signed']].drop_duplicates(ignore_index=True)
        
    def get_xprism_stake_amount(self, row):
        if(row['1_action']=='redeem_xprism'):
            return -row['amount']/1000000
        if(row['1_action']=='mint_xprism'):
            return row['1_amount']/1000000
    def get_prism_stake_amount(self, row):
        if(row['1_action']=='redeem_xprism'):
            return row['prism_queued']/1000000
        if(row['1_action']=='mint_xprism'):
            return -row['0_amount']/1000000
    
    def parse_prism_staking(self):
        self.prism_stake_df = self.standard_parse(self.prism_stake_df)
        prism_stake_df = self.prism_stake_df
        prism_stake_df['xprism_amount_signed'] = prism_stake_df.apply(self.get_xprism_stake_amount,axis=1)
        prism_stake_df['prism_amount_signed'] = prism_stake_df.apply(self.get_prism_stake_amount,axis=1)
        prism_stake_df['user'] = prism_stake_df['from_']
        prism_stake_df['action'] = prism_stake_df['1_action']
        self.prism_stake_df = prism_stake_df[['block_timestamp', 'hr', 'user',
               'tx_id', 'action', 'amount', 'prism_queued', 'day',
               'xprism_amount_signed', 'prism_amount_signed']].drop_duplicates(ignore_index=True)

    def write_to_csv(self):
        self.transfers_df.to_csv(f'{self.path_to_data}/transfers.csv')
        self.transfers_xprism_df.to_csv(f'{self.path_to_data}/transfers_xprism.csv')
        self.boost_activation_df.to_csv(f'{self.path_to_data}/boost_activation.csv')
        self.prism_forge_df.to_csv(f'{self.path_to_data}/prism_forge.csv')
        self.amps_bound_unbound_df.to_csv(f'{self.path_to_data}/amps_bound_unbound.csv')
        self.prism_stake_df.to_csv(f'{self.path_to_data}/prism_stake.csv')
        
    def parse(self):
        self.parse_transfers()
        self.parse_transfers_xprism()
        self.parse_forge_withdrawals()
        self.parse_amps_bonding()
        self.parse_prism_staking()
            
    

swaps_dp = SwapsDataProvider(claim, get_url, swaps_config, '../data/')
swaps_dp.load()
swaps_dp.parse()
#prism_dp.write_to_csv()

In [385]:
prism_dp = PrismBalanceDataProvider(claim, get_url, config, '../data/prism')
prism_dp.load()
prism_dp.write_to_csv()
prism_dp.parse()

In [386]:
df = prism_dp.prism_forge_df
df

,block_timestamp,tx_id,user,amount,day,amount_signed
0,2022-03-15 19:44:45,A5719FA36F3457BEE39B284A09F56987491FE5577FC7661C68F5DB1265F618AE,terra17ruvtyurpdt0qqcn47qduhlnp0uf6ln38u2kll,296.056399,2022-03-15,296.056399
1,2022-03-12 03:28:14,7F1A5617288F8F3C6FA83EA12578BFACD57725E9D315C09E5A1363F67FE07706,terra1dha0256a5lkhhg2p309jvypmq3w55z82rjxzyd,1776.338395,2022-03-12,1776.338395
2,2022-02-11 02:52:32,FD94B05377A8F688D5DDFBE195E29084ACE35FC8456421AA4150148FD486E3B1,terra1jmcjjexrn0anj2uyhwpxtjpue66c46tpjxl2c2,2960.563993,2022-02-11,2960.563993
3,2022-02-11 05:24:27,8170738FB7FB750A06A10143E28DA4C22486E8760BB99E99F50BE6A0497EE575,terra1k5hwkaar9snqzp7xadg536as678hp7ar2n9nah,296.056399,2022-02-11,296.056399
4,2022-02-11 18:45:38,9FC5FE3BD854ADA8CB26EEF99292A809492EF39B5CA4CFB7517B76494AA70E9E,terra185k3tqa38cjzmr26zt4c4hce2c9vyh82ngacal,266.450759,2022-02-11,266.450759
...,...,...,...,...,...,...
4997,2022-02-07 05:12:59,02F075B19643EBBA53D0A52E047BB640E759726CEB85F78FF00D4CDB90123638,terra19x8cw35dvsa473nq9uevgy0n9pjqe8mlper89a,592.112798,2022-02-07,592.112798
4998,2022-02-07 19:41:19,D8A2B4BC9A34EEF69CD3B08B1E2892C2D23CD9435F0E1FC06708A9257557F6A4,terra1ly3n43pd93s3gpzv7lfr7dasvfa06jmtgg2pm4,444.084598,2022-02-07,444.084598
4999,2022-02-07 09:36:44,0D04C388DD422F7568D5824B7A6FD23CD816DEF44C298815FACDE91144239E37,terra1l4rtpxxlm30rcmaevv0r8wxcejwd7ccqau9d6u,207.239479,2022-02-07,207.239479
5000,2022-02-07 20:03:34,FA07117093FFA962EF3E933BFC702175D0218F9A072BB1BD109EC8909BA2B8B4,terra1pmrn2cn39tny8gwdz8prfqpxt8lapmwn7r9ag3,5921.216163,2022-02-07,5921.216163


In [313]:
swaps_dp.swaps_single[swaps_dp.swaps_single.user=='terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q']\
                .groupby(['day','asset']).amount_signed.sum().reset_index()\
                .pivot(columns='asset',values='amount_signed',index='day').fillna(0).cumsum()

asset,LUNA,PRISM,UST,cLUNA,pLUNA,xPRISM
day,,,,,,
2022-02-14,0.000000,0.000000,0.0,0.000000,-7.976801,566.828750
2022-02-16,0.000000,0.000000,-5.0,0.000000,-7.976801,579.628825
2022-02-17,0.000000,0.000000,-20.0,0.000000,-7.976801,622.683202
2022-02-22,0.000000,0.000000,-35.0,0.000000,-7.976801,672.593408
2022-02-23,-5.000000,-31.687593,-35.0,5.114376,-7.976801,704.606639
2022-02-24,-15.000000,-31.687593,-35.0,15.347777,-7.976801,704.606639
2022-02-25,-16.000000,-31.687593,-35.0,15.347777,-2.734001,704.606639
2022-02-26,-16.500000,-31.687593,-35.0,15.347777,-0.022715,704.606639
2022-03-04,-22.524349,-31.687593,-35.0,21.577678,-0.022715,704.606639


**Prism stake**

In [282]:
prism_dp.prism_stake_df[prism_dp.prism_stake_df.user.isin(['terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv','terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q'])]


,block_timestamp,hr,user,tx_id,action,amount,prism_queued,day,xprism_amount_signed,prism_amount_signed


**AMPs**

In [223]:
prism_dp.amps_bound_unbound_df[prism_dp.amps_bound_unbound_df.user.isin(['terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv','terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q'])]


,block_timestamp,user,from_,hr,msg_index,to_,tx_id,action,amount,day,amount_signed
1836,2022-02-28 18:35:13,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,2022-02-28 18:00:00.000,0,terra1pa4amk66q8punljptzmmftf6ylq3ezyzx6kl9m,33E962D4919B200AB41F7D2AC28EAAF4C3A6C1F1D7E279D41A1DEB64BBC35234,bond,9.563402,2022-02-28,-9.563402
6199,2022-02-23 18:41:46,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,2022-02-23 18:00:00.000,0,terra1pa4amk66q8punljptzmmftf6ylq3ezyzx6kl9m,A5304009F2252072CC888E8136C5E029D5E410A0285AA77E573400422E144047,bond,1474.884278,2022-02-23,-1474.884278
7465,2022-02-23 18:45:34,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,2022-02-23 18:00:00.000,0,terra1pa4amk66q8punljptzmmftf6ylq3ezyzx6kl9m,E4907CD80E944669F81E3CCAF29059CB53C3AFCBC5742C8A5B5AC087D0AF5EE6,bond,137.777889,2022-02-23,-137.777889


**xPRISM transfers**

In [175]:
prism_dp.all_transfers_xprism[prism_dp.all_transfers_xprism.user=='terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q']

,block_timestamp,tx_id,user,amount,day,amount_signed
109,2022-02-06 19:58:46,A438D3042A5F940F90A0360E7A4EB44C7A7B9FCE8B8B3CFD24E6C19D4CB55F4B,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,470.729674,2022-02-06,-470.729674
44,2022-02-15 19:33:01,8CF7468DE489FFBEB2397A6BA58F746A623FAFF3C0131CA19C27BC1B6A9A9A25,terra1uh37lsydrup8vqvvttd53qwj93ft9x7572g62q,566.828750,2022-02-15,-566.828750


In [177]:
prism_dp.all_transfers_xprism[prism_dp.all_transfers_xprism.user=='terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv']

,block_timestamp,tx_id,user,amount,day,amount_signed
109,2022-02-06 19:58:46,A438D3042A5F940F90A0360E7A4EB44C7A7B9FCE8B8B3CFD24E6C19D4CB55F4B,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,470.729674,2022-02-06,470.729674
44,2022-02-15 19:33:01,8CF7468DE489FFBEB2397A6BA58F746A623FAFF3C0131CA19C27BC1B6A9A9A25,terra12j6p3tausehd7495as98vql74p0f9t2ahnafsv,566.828750,2022-02-15,566.828750
